In [14]:
# Load necessary libraries.
library(dplyr)
library(tidyr)
library(readxl)
library(clusterProfiler)
library(org.Mm.eg.db)
library(GO.db)

In [15]:
parse_data <- function() {
  dataset_file_path <- "./data/dataset.xlsx"
  dataset_sheet <- "Sheet1"

  full_data <- NULL
  df_2month_final <- data.frame()
  df_4month_final <- data.frame()
  df_6month_final <- data.frame()

  # Read in Excel file dataset.
  tryCatch({
    # --- Step 1: Load in all the data from the Excel file ---
    cat("Attempting to read Excel file:", dataset_file_path, ", 
      Sheet:", dataset_sheet, "\n")

    if (!file.exists(dataset_file_path)) {
      stop(paste("File not found at the specified path:", dataset_file_path, 
                 "Current R working directory is:", getwd()))
    }

    full_data <- read_excel(dataset_file_path, sheet = dataset_sheet,
                            col_types = "text", .name_repair = "minimal")
    cat("Excel file read successfully (all columns initially as text).\n")
    cat("Number of columns in full_data after reading:", ncol(full_data), "\n")

    # --- Step 2: Split the data immediately into three dataframes ---
    cat("--- Splitting data into 2-month, 4-month, and 6-month raw dataframes ---\n") #nolint
    cols_2month_indices <- 1:8      # A-H
    cols_4month_indices <- 10:17    # J-Q
    cols_6month_indices <- 19:26    # S-Z

    # Helper function to safely select columns.
    safe_select_cols <- function(df, col_indices, df_name_desc) {
      max_col_needed <- suppressWarnings(max(col_indices, na.rm = TRUE))
      actual_cols_in_df <- ncol(df)

      cat("For", df_name_desc, ": Attempting to select columns",
          min(col_indices), "to", max(col_indices),
          "\n\tMax needed:", max_col_needed, "\n\tActual cols in df:",
          actual_cols_in_df, "\n")

      if (actual_cols_in_df >= max_col_needed && length(col_indices) > 0
          && all(col_indices > 0)) {
        valid_indices <- col_indices[col_indices <= actual_cols_in_df]
        if (length(valid_indices) == length(col_indices)) {
          selected_df <- df %>% dplyr::select(all_of(valid_indices))
          cat(df_name_desc, "created with columns:", 
              paste(names(selected_df), collapse = ", "), "\n")
          return(as.data.frame(selected_df))
        } else {
          cat("Warning: Not all specified column indices for",
              df_name_desc, "are valid.\n\tMax index: ", max_col_needed,
              ", available cols: ", actual_cols_in_df, ".\n")
          return(data.frame())
        }
      } else {
        cat("Warning: Cannot create", df_name_desc,
            "- insufficient columns or invalid indices.\n\tNeeds up to",
            max_col_needed, "columns, available:", actual_cols_in_df, ".\n")
        return(data.frame())
      }
    }

    df_2month <- safe_select_cols(full_data, cols_2month_indices,
                                  "Raw 2-Month Dataframe")
    df_4month <- safe_select_cols(full_data, cols_4month_indices,
                                  "Raw 4-Month Dataframe")
    df_6month <- safe_select_cols(full_data, cols_6month_indices,
                                  "Raw 6-Month Dataframe")

    # --- Step 3: Filter out rows with entries "" or NA for each dataframe ---
    cat("--- Filtering completely blank rows from each dataframe ---\n")

    is_row_entirely_blank <- function(row_vector) {
      all(is.na(row_vector) | trimws(as.character(row_vector)) == "")
    }

    filter_blank_rows <- function(df, df_name) {
      if (nrow(df) > 0 && ncol(df) > 0) {
        rows_to_keep <- !apply(df, 1, is_row_entirely_blank)
        cat("Filtering blank rows for:", df_name,"\n\tOriginal rows:",
            nrow(df), "\n\tRows after filtering:", sum(rows_to_keep), "\n")
        return(df[rows_to_keep, , drop = FALSE])
      } else {
        cat("Skipping blank row filtering for:", df_name,
            "(0 rows or 0 columns).\n")
        return(df)
      }
    }

    df_2month_final <- filter_blank_rows(df_2month, "2-Month Data")
    df_4month_final <- filter_blank_rows(df_4month, "4-Month Data")
    df_6month_final <- filter_blank_rows(df_6month, "6-Month Data (post-shift)")

    # --- Step 4: Print out final dimensions ---
    cat("--- Final Data Dimensions (after all cleaning and filtering) ---\n")
    cat("   2-Month Data (df_2month_final): ",
        paste(dim(df_2month_final), collapse = " x "), "\n")
    cat("   4-Month Data (df_4month_final): ",
        paste(dim(df_4month_final), collapse = " x "), "\n")
    cat("   6-Month Data (df_6month_final): ",
        paste(dim(df_6month_final), collapse = " x "), "\n")

    cat("--- Data parsing and cleaning complete. Returning dataframes as a list. ---\n") #nolint
    return(list(
      df_2month = df_2month_final,
      df_4month = df_4month_final,
      df_6month = df_6month_final
    ))
  }, error = function(e) {
    cat("An error occurred within the parse_data function.\n")
    cat("Specific error message from R (conditionMessage(e)):\n",
        conditionMessage(e), "\n")
    cat("Full error object (e):\n")
    print(e)
    stop(
         paste("Stopping due to error in parse_data function. 
               Original error was:", conditionMessage(e)))
  })
}

In [16]:
processed_data <- parse_data()
if (!is.null(processed_data)) {
  data_2_month <- processed_data$df_2month
  data_4_month <- processed_data$df_4month
  data_6_month <- processed_data$df_6month

  cat("\n--- Head of Unpacked 2-Month Data ---\n")
  print(head(data_2_month))

  cat("\n--- Head of Unpacked 4-Month Data ---\n")
  print(head(data_4_month))

  cat("\n--- Head of Unpacked 6-Month Data ---\n")
  print(head(data_6_month))
} else {
  cat("parse_data() did not return any parsed data, 
      check for errors above.\n")
}

Attempting to read Excel file: ./data/dataset.xlsx , 
      Sheet: Sheet1 
Excel file read successfully (all columns initially as text).
Number of columns in full_data after reading: 27 
--- Splitting data into 2-month, 4-month, and 6-month raw dataframes ---
For Raw 2-Month Dataframe : Attempting to select columns 1 to 8 
	Max needed: 8 
	Actual cols in df: 27 
Raw 2-Month Dataframe created with columns: baseMean, log2FoldChange, FC, lfcSE, pvalue, padj, Gene, threshold 
For Raw 4-Month Dataframe : Attempting to select columns 10 to 17 
	Max needed: 17 
	Actual cols in df: 27 
Raw 4-Month Dataframe created with columns: baseMean, log2FoldChange, FC, lfcSE, pvalue, padj, Gene, threshold 
For Raw 6-Month Dataframe : Attempting to select columns 19 to 26 
	Max needed: 26 
	Actual cols in df: 27 
Raw 6-Month Dataframe created with columns: baseMean, log2FoldChange, FC, lfcSE, pvalue, padj, Gene, threshold 
--- Filtering completely blank rows from each dataframe ---
Filtering blank rows fo

In [23]:
map_genes_to_go_cc <- function(gene_symbols_list, organism_db) {
  cat("--- Mapping gene symbols to Entrez IDs ---\n")
  entrez_ids <- mapIds(
    organism_db, keys = gene_symbols_list, column = "ENTREZID",
    keytype = "SYMBOL", multiVals = "first"
  )
  gene_map <- data.frame(
    GeneSymbol = names(entrez_ids), ENTREZID = entrez_ids,
    stringsAsFactors = FALSE
  )
  gene_map <- gene_map %>% dplyr::filter(!is.na(ENTREZID))

  if (nrow(gene_map) == 0) {
    cat("\tNo Entrez IDs found for the provided symbols. Returning empty map.\n") #nolint
    return(
      data.frame(GeneSymbol = character(), ENTREZID = character(),
      GO_Term = character()) # nolint
    )
  }
  cat(paste0("\tFound Entrez IDs for ", nrow(gene_map), " symbols.\n"))

  cat("--- Mapping Entrez IDs to GO IDs (Cellular Components only) ---\n")
  go_data <- AnnotationDbi::select(
    organism_db, keys = gene_map$ENTREZID,
    keytype = "ENTREZID", columns = c("GO", "ONTOLOGY")
  )
  go_cc_data <- go_data %>% 
    dplyr::filter(
      ONTOLOGY == "CC" & !is.na(GO)
    ) %>% 
    dplyr::select(ENTREZID, GO_ID = GO)
  go_ids <- unique(go_cc_data$GO_ID)

  if (length(go_ids) > 0) {
    cat("--- Mapping GO IDs to GO Term Names ---\n")
    go_terms_map <- AnnotationDbi::select(
      GO.db, keys = go_ids, keytype = "GOID", columns = "TERM"
    )
    go_terms_map <- go_terms_map %>% dplyr::rename(GO_ID = GOID, GO_Term = TERM)
    go_cc_data <- dplyr::left_join(go_cc_data, go_terms_map, by = "GO_ID")

    cat("--- Creating final Gene-to-Cellular Component map ---\n")
    final_map <- dplyr::left_join(gene_map, go_cc_data, by = "ENTREZID")
    final_map <- final_map %>%
      dplyr::filter(
        !is.na(GO_ID) & !is.na(GO_Term)
      ) %>%
      dplyr::select(
        GeneSymbol, ENTREZID, GO_Term
      ) %>%
      dplyr::distinct()

    cat("--- Mapping Complete ---\n")
    return(final_map) # nolint
  } else {
    cat("--- No GO Cellular Component terms found. Returning empty map. ---\n")
    return(data.frame(GeneSymbol = character(), ENTREZID = character(), GO_Term = character())) # nolint
  }
}

In [24]:
get_go_term_counts <- function(mapping_df) {
  cat("--- Counting gene markers per cellular component ---\n")

  if (exists("mapping_df") && nrow(mapping_df) > 0) {
    go_term_counts <- mapping_df %>% 
      group_by(GO_Term) %>% 
      summarise(
        GeneCount = n_distinct(GeneSymbol),
        .groups = "drop"
      ) %>% 
      arrange(desc(GeneCount))

    cat("--- Gene counts dataframe created ---\n")
    return(go_term_counts) # nolint
  } else {
    cat("--- Cannot create counts: Input dataframe is empty or does not exist. ---\n") # nolint
    return(data.frame(GO_Term = character(), GeneCount = integer())) # nolint
  }
}

In [25]:
# --- Step 1: Extract gene symbols for each dataframe ---
cat("--- Extracting gene symbols from 2, 4, and 6 months dataframes ---\n")
genes_2_month <- data_2_month$Gene
genes_4_month <- data_4_month$Gene
genes_6_month <- data_6_month$Gene

cat("\n\n--- Processing 2-Month Gene Data ---\n")
final_2_month_map <- map_genes_to_go_cc(genes_2_month, org.Mm.eg.db)
go_term_counts_2m <- get_go_term_counts(final_2_month_map)
cat("--- Top 10 most populated cellular components (2-Month data) ---\n")
print(head(go_term_counts_2m, 10))

cat("\n\n--- Processing 4-Month Gene Data ---\n")
final_4_month_map <- map_genes_to_go_cc(genes_4_month, org.Mm.eg.db)
go_term_counts_4m <- get_go_term_counts(final_4_month_map)
cat("--- Top 10 most populated cellular components (4-Month data) ---\n")
print(head(go_term_counts_4m, 10))

cat("\n\n--- Processing 6-Month Gene Data ---\n")
final_6_month_map <- map_genes_to_go_cc(genes_6_month, org.Mm.eg.db)
go_term_counts_6m <- get_go_term_counts(final_6_month_map)
cat("--- Top 10 most populated cellular components (6-Month data) ---\n")
print(head(go_term_counts_6m, 10))

--- Extracting gene symbols from 2, 4, and 6 months dataframes ---


--- Processing 2-Month Gene Data ---
--- Mapping gene symbols to Entrez IDs ---


'select()' returned 1:1 mapping between keys and columns



	Found Entrez IDs for 11750 symbols.
--- Mapping Entrez IDs to GO IDs (Cellular Components only) ---


'select()' returned 1:many mapping between keys and columns



--- Mapping GO IDs to GO Term Names ---


...

	Found Entrez IDs for 11143 symbols.
--- Mapping Entrez IDs to GO IDs (Cellular Components only) ---


'select()' returned 1:many mapping between keys and columns



--- Mapping GO IDs to GO Term Names ---


'select()' returned 1:1 mapping between keys and columns



--- Creating final Gene-to-Cellular Component map ---
--- Mapping Complete ---
--- Counting gene markers per cellular component ---
--- Gene counts dataframe created ---
--- Top 10 most populated cellular components (6-Month data) ---
# A tibble: 10 × 2
   GO_Term               GeneCount
   <chr>                     <int>
 1 cytoplasm                  3798
 2 nucleus                    3617
 3 cytosol                    2858
 4 nucleoplasm                2436
 5 plasma membrane            2423
 6 membrane                   1399
 7 mitochondrion              1298
 8 cellular_component          838
 9 endoplasmic reticulum       804
10 Golgi apparatus             781
